In [1]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import math
import ipywidgets as widgets
from ipywidgets import VBox, HBox, interactive
import math
import matplotlib.image as img
import cv2
from IPython.display import clear_output, display
from tkinter import Tk, filedialog
from datetime import datetime
import seaborn as sns
import imageio

This notebook consists of the following classes:
  
    
    class main
    class subjects
    class session
    class bottom_cam
    class maze corner
    class clean_df
    class df_functions
    class stats
    
    class gui
        class main_gui
        class subject_gui
        class clean_df_gui
        class select_functions
        
    class maze_corner_gui       
    class stats_gui


Name all of your .csv/.mp4/.avi files strictly like this: 

                        210_F1-83_220518_OTT_bottom
                        211_F3-04_211123_OTR_top

                        Line_ID_Date_Paradigm_Camera   
                    
Your framerate should not have decimal places.

In [2]:
class main: 
    """Main object contains overview over all files, subjects and common variable over a folder of files as specified by the path."""
    l_maze_corners_bc = []
    l_maze_corners_tc = []

    def __init__(self, path = "PATH", dict_cams_used = {"bottom_cam": True, "top_cam": False, "side_cam": False}, gui = False):
        #creates common variables
        main.path = path
        main.dict_cams_used = dict_cams_used
        main.gui = gui
        if main.path != "PATH":
            self.all_information_given()
        
    def all_information_given(self):
        #creates objects for all of the files in the chosen folder
        main.l_session_IDs = np.unique(np.array([file for file in set ([elem[15:-59] for elem in os.listdir(self.path) if elem.endswith('CatwalkMay25shuffle1_90000_filtered.csv')])])).tolist()
        main.l_sessions = [session(session_ID) for session_ID in self.l_session_IDs]
        main.l_subject_IDs = np.unique(np.array([session.subject_ID for session in self.l_sessions])).tolist()
        main.l_subjects = [subject(subject_ID) for subject_ID in self.l_subject_IDs]
            
    def subjects_to_groups(self, l_groups):
        #provides each subject a subject_ID as in subject_group_dict
        if main.all_files_there:
            main.l_groups = l_groups
            if main.gui==False:
                #branch that will be used if gui-less usage is chosen
                if self.dict_cams_used["bottom_cam"]:
                    self.get_maze_corners()

    def get_maze_corners(self):
        #if top or bottom cam are used, this function creates the maze_corner annotation
        if self.all_files_there & (1>3):
            main.l_maze_corners = [session.bc.mc for session in self.l_sessions]
            if gui.displayed == False:
                display(gui.main_tab)
                gui.displayed = True
            maze_corner_gui()
        else:
            self.get_processed_dfs()
                   
    def get_processed_dfs(self, DLC_likelihood_threshold = 0.75):
        #processes the dataframes
        main.dict_bodyparts = {}#wird zukünftig ersetzt durch class bodypart 
        if self.dict_cams_used["bottom_cam"]:
            for session in self.l_sessions:
                session.bc.processed_df = clean_df(session.bc, DLC_likelihood_threshold).df 
        
    def execute_functions(self, dict_selected_functions = {"Anxiety": False, "Parkinson": False, "Catwalk": True}):
        #calculates the target variables as chosen in dict_selected_functions
        self.get_col_in_master_df_and_main_dataframe(dict_selected_functions)
        for subject in self.l_subjects:
            for session in subject.l_sessions:
                session.master_df = df_functions(subject, session, dict_selected_functions).master_df
                self.append_session_to_d_data(subject, session, dict_selected_functions)
        if gui.displayed == False:
            display(gui.main_tab)
            gui.displayed = True
        stats_gui(dict_selected_functions)
        
    def get_col_in_master_df_and_main_dataframe(self, dict_selected_functions):
        #creates the master and main dataframe
        main.d_data = {'subject_ID': [], 'group_ID': [], 'paradigm': [], }
        main.col_in_master_df = [('subject_ID', ('subject_ID', '')),
                    ('group_ID', ('group_ID', '')),
                    ('paradigm', ('paradigm', '')),
                    ('time', ('time', '')),
                    ('exclude', ('all', 'exclude')), 
                    ('CenterOfGravity_x_norm_cm', ('CenterOfGravity', 'x_norm_cm')),
                    ('CenterOfGravity_y_norm_cm', ('CenterOfGravity', 'y_norm_cm')),                    
                    ('CenterOfGravity_rolling_speed_px_per_s', ('CenterOfGravity', 'rolling_speed_px_per_s'))]
        
        d_data_cat = {'mean_angle_secondfinger_hindpawright_hindkneeright' : [], 
                  'mean_angle_secondfinger_hindpawleft_hindkneeleft' : [], 
                  'mean_angle_hindpawright_hindkneeright_CoG': [], 
                  'mean_angle_hindpawleft_hindkneeleft_CoG': [], 
                  'mean_distance_hindpawrightsecondfinger_hindpawright': [], 
                  'mean_distance_hindpawleftsecondfinger_hindpawleft': [],
                  'mean_area_hindpawright': [], 
                  'mean_area_hindpawleft': [], } 
        main.d_data = self.d_data | d_data_cat
        main.col_in_master_df.extend([("angle_secondfinger_hindpawright_hindkneeright", ("angle_secondfinger_hindpawright_hindkneeright", "")), 
                                      ("angle_secondfinger_hindpawleft_hindkneeleft", ("angle_secondfinger_hindpawleft_hindkneeleft", "")),
                                      ("angle_hindpawright_hindkneeright_CoG", ("angle_hindpawright_hindkneeright_CoG", "")),
                                      ("angle_hindpawleft_hindkneeleft_CoG", ("angle_hindpawleft_hindkneeleft_CoG", "")),
                                      ("distance_secondfinger_hindpawright", ("distance_secondfinger_hindpawright", "")),
                                      ("distance_secondfinger_hindpawleft", ("distance_secondfinger_hindpawleft", "")),
                                      ("area_HindPawRight", ("area_HindPawRight", "")),
                                      ("area_HindPawLeft", ("area_HindPawLeft", ""))])
           
    def append_session_to_d_data(self, subject, session, dict_selected_functions): 
        #appends data of each session to a dict containing all sessions
        wall_end_position = 35 #in cm

        self.d_data['subject_ID'].append(session.subject_ID)
        self.d_data['group_ID'].append(subject.group_ID)
        self.d_data['paradigm'].append(session.paradigm)         
                          
        self.d_data['mean_angle_secondfinger_hindpawright_hindkneeright'].append(session.master_df['angle_hindpawright_hindkneeright_CoG'].mean())
        self.d_data['mean_angle_secondfinger_hindpawleft_hindkneeleft'].append(session.master_df['angle_hindpawleft_hindkneeleft_CoG'].mean())
        self.d_data['mean_angle_hindpawright_hindkneeright_CoG'].append(session.master_df['angle_secondfinger_hindpawright_hindkneeright'].mean())
        self.d_data['mean_angle_hindpawleft_hindkneeleft_CoG'].append(session.master_df['angle_secondfinger_hindpawleft_hindkneeleft'].mean())
        self.d_data['mean_distance_hindpawrightsecondfinger_hindpawright'].append(session.master_df['distance_secondfinger_hindpawright'].mean())
        self.d_data['mean_distance_hindpawleftsecondfinger_hindpawleft'].append(session.master_df['distance_secondfinger_hindpawleft'].mean())
        self.d_data['mean_area_hindpawright'].append(session.master_df['area_HindPawRight'].mean())
        self.d_data['mean_area_hindpawleft'].append(session.master_df['area_HindPawLeft'].mean())

        main.dataframe = pd.DataFrame(data = self.d_data)
        
    def calculate_stats(self, output_path, l_selected_data_col_dict):
        #calls the stats class with inputs as chosen in the stats_gui
        for dict_stats in l_selected_data_col_dict:
            stats.total_count_stats(self, dict_stats["data_col"], dict_stats["independent_variable"], dict_stats["hue"], output_path)
            if "y_position" in dict_stats["data_col"]:
                stats.position_stats(self, dict_stats["data_col"], dict_stats["independent_variable"], dict_stats["hue"], output_path)
    
    def save(self):
        save_dict = {}
        #save_dict['all_files_there'] = self.all_files_there
        #save_dict['dict_bodyparts'] = self.dict_bodyparts
        save_dict['main_obj'] = self
        save_dict['dict_cams_used'] = self.dict_cams_used
        #save_dict['gui'] = self.gui
        #save_dict['l_groups'] = self.l_groups
        save_dict['l_maze_corners'] = self.l_maze_corners
        #save_dict['l_session_IDs'] = self.l_session_IDs
        save_dict['l_sessions'] = self.l_sessions
        #save_dict['l_subject_IDs'] = self.l_subject_IDs
        save_dict['l_subjects'] = self.l_subjects
        save_dict['path'] = self.path
        with open('data.pickle', 'wb') as f:
            pickle.dump(save_dict, f, pickle.HIGHEST_PROTOCOL)
        f.close()
        
    def safe(self):
        saved_dic = vars(main)
        print(type(saved_dic))
        with open('data.pickle', 'wb') as f:
            pickle.dump(saved_dic, f, pickle.HIGHEST_PROTOCOL)
        f.close()

In [3]:
class subject(main):
    """creates an object representing an individual subject"""
    def __init__(self, subject_ID):
        self.subject_ID = subject_ID
        self.l_sessions = [session for session in main.l_sessions if session.subject_ID == self.subject_ID]
        #testen
        self.l_paradigms = [session.paradigm for session in self.l_sessions]
        if len(self.l_paradigms) > 3:
            self.group_ID = "experimental"
        else:
            self.group_ID = "control"
        self.dict_date_paradigm = {session.date:session.paradigm for session in self.l_sessions}

In [4]:
class session(subject):
    """creates an object representing an individual session and checks, whether all the files are there"""
    def __init__(self, session_ID):
        self.session_ID = session_ID
        print(session_ID)
        self.paradigm = "None"
        if 'prä-OP1' in self.session_ID:
            self.paradigm = 'prä-OP1'
        elif 'prä-OP2'  in self.session_ID:
            self.paradigm  = 'prä-OP2'
        elif 'prä-OP3'  in self.session_ID:
            self.paradigm = 'prä-OP3'
        elif 'post-OP'  in self.session_ID:
            self.paradigm = 'post-OP'
        elif 'post1xIgG'  in self.session_ID or 'post-1xIgG' in self.session_ID:
            self.paradigm = 'post1xIgG'
        elif 'post3xIgG'  in self.session_ID:
            self.paradigm = 'post3xIgG'
        elif 'post6xIgG'  in self.session_ID:
            self.paradigm = 'post6xIgG'
        if self.paradigm == "None":
            print("File for session {} has to be renamed!".format(self.session_ID))
        self.date = datetime.strptime(self.session_ID[-6:], '%y%m%d')
        self.subject_ID = self.session_ID[0:6] 
        bc_csv = self.session_ID in set([elem[15:-59] for elem in os.listdir(main.path) if elem.endswith('CatwalkMay25shuffle1_90000_filtered.csv')])
        bc_mp4 = self.session_ID in set([elem[15:-4] for elem in os.listdir(main.path) if elem.endswith('.mp4')]) 
        
        
        main.all_files_there = True
        if bc_csv == False:
            print("Missing .csv file for session {} in {}!".format(self.session_ID, main.path))
            main.all_files_there = False
        elif bc_mp4 == False:
            print("Missing .mp4 file for session {} in {}!".format(self.session_ID, main.path))
            main.all_files_there = False
        mp4_file_list = [elem for elem in os.listdir(main.path) if elem.endswith('.mp4')]
        for n in range(len(mp4_file_list)):
            if session_ID in mp4_file_list[n]:
                bc_video_filename = main.path+mp4_file_list[n]
        csv_file_list = [elem for elem in os.listdir(main.path) if elem.endswith('CatwalkMay25shuffle1_90000_filtered.csv')]
        for n in range(len(csv_file_list)):
            if session_ID in csv_file_list[n]:
                bc_csv_filename = csv_file_list[n]
        bc_df = pd.read_csv(main.path + bc_csv_filename, skiprows=1, index_col=0, header=[0, 1])
        self.bc = bottom_cam(self.session_ID, bc_df, bc_video_filename)


In [5]:
class bottom_cam(session):
    """creates an object containing the data for bottom cam recording modality"""
    def __init__(self, session_ID, df, video_file_name):
        self.session_ID = session_ID
        self.df = df
        self.video_path = video_file_name 
        self.mc = maze_corners(self.video_path)
        video = imageio.get_reader(self.video_path,  'ffmpeg')
        self.framerate = video.get_meta_data()["fps"]
        self.cam = "bottom_cam"

In [6]:
class clean_df(session):
    """creates an object that returns the cleaned dataframe after processing several functions"""
    maze_length_in_cm = 50
    
    def __init__(self, session_cam_object, threshold):        
        self.threshold = threshold
        self.framerate = session_cam_object.framerate
        self.results = session_cam_object.mc.results
        df = session_cam_object.df
        self.l_bodyparts = [elem[0] for elem in df.columns[::3]]
        main.dict_bodyparts[session_cam_object.cam] = self.l_bodyparts
        df = self.get_time(df)
        df = self.identify_duplicates(df)
        df[('all', 'exclude')] = False
        df = self.exclude_frames(df)
        if "CenterOfGravity" not in self.l_bodyparts:
            df = self.get_center_of_gravity(df)
        df = self.normalize_coordinates(df)
        self.df = df
        
    def get_time(self, df):
        #appends the dataframe with an column time, calculated by index and framerate
        df['time'] = np.NaN
        df['time'] = df['EarRight'].index/self.framerate
        return df
    
    def identify_duplicates(self, df):
        #checks for possible duplicates made by the DLC network and excludes them
        l_indices = list(df.index)
        l_unique_indices = list(set(l_indices))

        if len(l_indices) != len(l_unique_indices):
            l_duplicates = []
            for index in l_unique_indices:
                if l_indices.count(index) > 1:
                    l_duplicates.append(index)
            df.loc[l_duplicates, ('all', 'exclude')] = True

        return df

    def exclude_frames(self, df):
        #excludes frames, where the likelihood is below a certain threshold
        for bodypart in self.l_bodyparts:
            df.loc[:, (bodypart, 'exclude')] = False
            df.loc[df[bodypart]['likelihood'] < self.threshold, (bodypart, 'exclude')] = True
            df.loc[df[('all', 'exclude')] == True, (bodypart, 'exclude')] = True
        return df
    
    def rotate(self, xy, theta):
        cos_theta, sin_theta = math.cos(theta), math.sin(theta)
        return (xy[0] * cos_theta - xy[1] * sin_theta, xy[0] * sin_theta + xy[1] * cos_theta)

    def translate(self, xy, offset):
        return xy[0] + offset[0], xy[1] + offset[1]


    def normalize_coordinates(self, df):
        #uses the functions rotate and translate to normalize the coordinates
        length = self.results['length']
        width = self.results['width']
        offset_to_standard = (-self.results['offset_x'], -self.results['offset_y'])
        offset_from_standard = (self.results['offset_x'], self.results['offset_y'])
        theta_to_standard = -self.results['theta']


        length_in_px = length
        cm_per_px = self.maze_length_in_cm/length_in_px

        for bodypart in self.l_bodyparts:
            df[(bodypart, 'x_norm')] = self.translate(self.rotate((df[(bodypart,'x')], df[(bodypart,'y')]), theta_to_standard), offset_to_standard)[0]
            df[(bodypart, 'y_norm')] = self.translate(self.rotate((df[(bodypart,'x')], df[(bodypart,'y')]), theta_to_standard), offset_to_standard)[1]
            df[(bodypart, 'x_norm_cm')] = 3 - (df[(bodypart, 'x_norm')] * cm_per_px)
            df[(bodypart, 'y_norm_cm')] = 50 - (df[(bodypart, 'y_norm')] * cm_per_px)
        return df   

    
    def get_center_of_gravity(self, df):
        #calculates the center of gravity, if it is not labeled in DLC
        df.loc[(df[('all', 'exclude')] == False) & (df[('EarRight', 'exclude')] == False) & (df[('EarLeft', 'exclude')] == False) & (df[('TailBase', 'exclude')] == False), 
           ('CenterOfGravity', 'x')] = (df.loc[df[('all', 'exclude')] == False, ('EarRight', 'x')] + 
                                          df.loc[df[('all', 'exclude')] == False, ('EarLeft', 'x')] + 
                                          df.loc[df[('all', 'exclude')] == False, ('TailBase', 'x')]) / 3

        df.loc[(df[('all', 'exclude')] == False) & (df[('EarRight', 'exclude')] == False) & (df[('EarLeft', 'exclude')] == False) & (df[('TailBase', 'exclude')] == False), 
           ('CenterOfGravity', 'y')] = (df.loc[df[('all', 'exclude')] == False, ('EarRight', 'y')] + 
                                          df.loc[df[('all', 'exclude')] == False, ('EarLeft', 'y')] + 
                                          df.loc[df[('all', 'exclude')] == False, ('TailBase', 'y')]) / 3

        df[('CenterOfGravity', 'exclude')] = False
        df.loc[(df[('CenterOfGravity', 'x')].isnull()) | (df[('CenterOfGravity', 'y')].isnull()), ('CenterOfGravity', 'exclude')] = True

        self.l_bodyparts.append('CenterOfGravity')
        return df

In [13]:
class df_functions(bottom_cam):
    """class that returns the dataframe appended with the chosen target variables after calling several functions and combines different recording modalities"""
    immobility_threshold = 16

    min_freezing_duration = 1

    TIME_OF_GAIT_BEFORE_DISRUPT = 0.5
    TARGET_TIME_GAIT_DISRUPTION = 0.2
    
    def __init__(self, subject, session, dict_selected_functions):
        #loops through the functions
        self.session = session
        self.subject = subject
        self.dict_selected_functions = dict_selected_functions
        df = session.bc.processed_df
        self.l_bodyparts = main.dict_bodyparts["bottom_cam"]
        self.framerate = session.bc.framerate

        df = self.get_speed_and_rolling_speed(df)
        df = self.get_immobility(df)
        df = self.get_bodypart_angles(df)
        df = self.get_distance_secondfinger_pawbase(df)
        df = self.get_paw_area(df)
            
        self.combined_df = self.add_metadata(df)
        self.master_df = self.get_master_df()
                
    def get_speed_and_rolling_speed(self, df):
        #calcualtes speed for the bodyparts
        for bodypart in self.l_bodyparts:
            df[(bodypart, 'speed_px_per_s')] = np.NaN
            df[(bodypart, 'rolling_speed_px_per_s')] = np.NaN

            # Limitation: since we have to exclude some frames, these calculations are not made frame by frame (yet for most)
            df.loc[(df[('all', 'exclude')] == False) & (df[(bodypart, 'exclude')] == False), (bodypart, 'speed_px_per_s')] = (((df.loc[(df[('all', 'exclude')] == False) & (df[(bodypart, 'exclude')] == False), (bodypart, 'x')].diff()**2                                                                                                        + df.loc[(df[('all', 'exclude')] == False) & (df[(bodypart, 'exclude')] == False), (bodypart, 'y')].diff()**2)**(1/2)) 
                                                                                                                             / df.loc[(df[('all', 'exclude')] == False) & (df[(bodypart, 'exclude')] == False), 'time'].diff())
            df.loc[(df[('all', 'exclude')] == False) & (df[(bodypart, 'exclude')] == False), (bodypart, 'rolling_speed_px_per_s')] = df.loc[df[('all', 'exclude')] == False, (bodypart, 'speed_px_per_s')].rolling(5, min_periods=3, center=True).mean()
        return df
    
    def get_immobility(self, df):
        #checks for immobility
        for bodypart in self.l_bodyparts:
            # create 'immobility' column and set base value to false
            df.loc[ :, (bodypart, 'immobility')] = False
            df.loc[df[(bodypart,'rolling_speed_px_per_s')] < self.immobility_threshold, (bodypart, 'immobility')] = True
        return df
        
    def add_metadata(self, df):
        #adds metadata to the dataframe
        df['subject_ID'] = self.subject.subject_ID
        df['group_ID'] = self.subject.group_ID
        df['DateOfRecording'] = self.session.date
        df['paradigm'] = self.session.paradigm
        return df
    
    def get_master_df(self):
        #combines all the information into a master_df as specified by col_in_precessed_df
        d_for_master_df = {}
        for key, col_in_processed_df in main.col_in_master_df:
            d_for_master_df[key] = self.combined_df[col_in_processed_df].values

        master_df = pd.DataFrame(data=d_for_master_df)
        return master_df
        
    def get_bodypart_angles(self, df):
        df['angle_secondfinger_hindpawright_hindkneeright'] = np.NaN
        df['angle_secondfinger_hindpawleft_hindkneeleft'] = np.NaN
        df['angle_hindpawright_hindkneeright_CoG'] = np.NaN
        df['angle_hindpawleft_hindkneeleft_CoG'] = np.NaN
        
        #HindPawRightSecondFinger
        cy = df.loc[:, ('HindPawRightSecondFinger', 'y_norm_cm')]
        cx = df.loc[:, ('HindPawRightSecondFinger', 'x_norm_cm')]
        #HindPawRight
        by = df.loc[:, ('HindPawRight', 'y_norm_cm')]
        bx = df.loc[:, ('HindPawRight', 'x_norm_cm')]
        #HindKneeRight
        ay = df.loc[:, ('HindKneeRight', 'y_norm_cm')]
        ax = df.loc[:, ('HindKneeRight', 'x_norm_cm')]
        #angle_secondfinger_hindpawright_hindkneeright
        df_copy = df.copy()
        df_copy.loc[:, ('angle_secondfinger_hindpawright_hindkneeright')] = self.getAngle(ax, ay, bx, by, cx, cy)
        print(df_copy.loc[(df[('all', 'exclude')] == False) & (df[('HindKneeRight', 'exclude')] == False) & (df[('HindPawRight', 'exclude')] == False) & (df[('HindPawRightSecondFinger', 'exclude')] == False), ('angle_secondfinger_hindpawright_hindkneeright')])
        df.loc[:, ('angle_secondfinger_hindpawright_hindkneeright')] = df_copy.loc[(df[('all', 'exclude')] == False) & (df[('HindKneeRight', 'exclude')] == False) & (df[('HindPawRight', 'exclude')] == False) & (df[('HindPawRightSecondFinger', 'exclude')] == False), ('angle_secondfinger_hindpawright_hindkneeright')]
        
        #HindPawRight
        cy = by
        cx = bx
        #HindKneeRight
        by = ay
        bx = ax
        #CoG
        ay = df.loc[:, ('CenterOfGravity', 'y_norm_cm')] 
        ax = df.loc[:, ('CenterOfGravity', 'x_norm_cm')] 
        #angle_hindpawright_hindkneeright_CoG      
        df_copy = df.copy()
        df_copy.loc[:, ('angle_hindpawright_hindkneeright_CoG')] = self.getAngle(ax, ay, bx, by, cx, cy)
        df.loc[:, ('angle_hindpawright_hindkneeright_CoG')] = df_copy.loc[(df[('all', 'exclude')] == False) & (df[('HindPawRight', 'exclude')] == False) & (df[('CenterOfGravity', 'exclude')] == False) & (df[('HindKneeRight', 'exclude')] == False), ('angle_hindpawright_hindkneeright_CoG')]
        print(df_copy.loc[(df[('all', 'exclude')] == False) & (df[('HindPawRight', 'exclude')] == False) & (df[('CenterOfGravity', 'exclude')] == False) & (df[('HindKneeRight', 'exclude')] == False), ('angle_hindpawright_hindkneeright_CoG')])
        
        #HindPawleftSecondFinger
        cy = df.loc[:, ('HindPawLeftSecondFinger', 'y_norm_cm')]
        cx = df.loc[:, ('HindPawLeftSecondFinger', 'x_norm_cm')]
        #HindPawleft
        by = df.loc[:, ('HindPawLeft', 'y_norm_cm')]
        bx = df.loc[:, ('HindPawLeft', 'x_norm_cm')]
        #HindKneeleft
        ay = df.loc[:, ('HindKneeleft', 'y_norm_cm')]
        ax = df.loc[:, ('HindKneeleft', 'x_norm_cm')]
        #angle_secondfinger_hindpawleft_hindkneeleft
        df_copy = df.copy()
        df_copy.loc[:, ('angle_secondfinger_hindpawleft_hindkneeleft')] = self.getAngle(ax, ay, bx, by, cx, cy)
        df.loc[:, ('angle_secondfinger_hindpawleft_hindkneeleft')] = df_copy.loc[((df[('all', 'exclude')] == False) & (df[('HindPawLeft', 'exclude')] == False) & (df[('HindKneeleft', 'exclude')] == False) & (df[('HindPawLeftSecondFinger', 'exclude')] == False), ('angle_secondfinger_hindpawleft_hindkneeleft'))]
        print(df_copy.loc[((df[('all', 'exclude')] == False) & (df[('HindPawLeft', 'exclude')] == False) & (df[('HindKneeleft', 'exclude')] == False) & (df[('HindPawLeftSecondFinger', 'exclude')] == False), ('angle_secondfinger_hindpawleft_hindkneeleft'))])
        
        #HindPawleft
        cy = by
        cx = bx
        #HindKneeleft
        by = ay
        bx = ax
        #CoG
        ay = df.loc[:, ('CenterOfGravity', 'y_norm_cm')] 
        ax = df.loc[:, ('CenterOfGravity', 'x_norm_cm')] 
        #angle_hindpawleft_hindkneeleft_CoG      
        df_copy = df.copy()
        df_copy.loc[:, ('angle_hindpawleft_hindkneeleft_CoG')] = self.getAngle(ax, ay, bx, by, cx, cy)
        df.loc[:, ('angle_hindpawleft_hindkneeleft_CoG')] = df_copy.loc[(df[('all', 'exclude')] == False) & (df[('HindPawLeft', 'exclude')] == False) & (df[('CenterOfGravity', 'exclude')] == False) & (df[('HindKneeleft', 'exclude')] == False), ('angle_hindpawleft_hindkneeleft_CoG')]
        print(df_copy.loc[(df[('all', 'exclude')] == False) & (df[('HindPawLeft', 'exclude')] == False) & (df[('CenterOfGravity', 'exclude')] == False) & (df[('HindKneeleft', 'exclude')] == False), ('angle_hindpawleft_hindkneeleft_CoG')])
        
        return df
    
    def getAngle(self, ax, ay, bx, by, cx, cy):
        ang = np.rad2deg(np.arctan2((cy-by) - (ay-by), (cx-bx) - (ax-bx)))
        ang = np.where(ang<0, 360+ang, ang)
        return ang
    
    def get_distance_secondfinger_pawbase(self, df):
        df['distance_secondfinger_hindpawright'] = np.NaN
        df['distance_secondfinger_hindpawleft'] = np.NaN
                  
        df_copy = df.copy()
        df_copy.loc[:, ('distance_secondfinger_hindpawright')] = np.sqrt((df.loc[:, ('HindPawRightSecondFinger', 'x_norm_cm')] - df.loc[:, ('HindPawRight', 'x_norm_cm')])**2 + (df.loc[:, ('HindPawRightSecondFinger', 'y_norm_cm')]- df.loc[:, ('HindPawRight', 'y_norm_cm')])**2)
        df.loc[:, ('distance_secondfinger_hindpawright')] = df_copy.loc[(df[('all', 'exclude')] == False) & (df[('HindPawRight', 'exclude')] == False) & (df[('HindPawRightSecondFinger', 'exclude')] == False), ('distance_secondfinger_hindpawright')]
        print(df_copy.loc[(df[('all', 'exclude')] == False) & (df[('HindPawRight', 'exclude')] == False) & (df[('HindPawRightSecondFinger', 'exclude')] == False), ('distance_secondfinger_hindpawright')])
        
        df_copy = df.copy()
        df_copy.loc[:, ('distance_secondfinger_hindpawleft')] = np.sqrt((df.loc[:, ('HindPawLeftSecondFinger', 'x_norm_cm')] - df.loc[:, ('HindPawLeft', 'x_norm_cm')])**2 + (df.loc[:, ('HindPawLeftSecondFinger', 'y_norm_cm')]- df.loc[:, ('HindPawLeft', 'y_norm_cm')])**2)
        df.loc[:, ('distance_secondfinger_hindpawleft')] = df_copy.loc[(df[('all', 'exclude')] == False) & (df[('HindPawLeft', 'exclude')] == False) & (df[('HindPawLeftSecondFinger', 'exclude')] == False), ('distance_secondfinger_hindpawleft')]
        print(df_copy.loc[(df[('all', 'exclude')] == False) & (df[('HindPawLeft', 'exclude')] == False) & (df[('HindPawLeftSecondFinger', 'exclude')] == False), ('distance_secondfinger_hindpawleft')])
        
        return df
        
    def get_paw_area(self, df):
        df['area_HindPawRight'] = np.NaN
        df['area_HindPawLeft'] = np.NaN
        
        #HindPawRightSecondFinger
        cy = df.loc[:, ('HindPawRightSecondFinger', 'y_norm_cm')]
        cx = df.loc[:, ('HindPawRightSecondFinger', 'x_norm_cm')]
        #HindPawRight
        by = df.loc[:, ('HindPawRight', 'y_norm_cm')]
        bx = df.loc[:, ('HindPawRight', 'x_norm_cm')]
        #HindPawRightFifthFinger
        ay = df.loc[:, ('HindPawRightFifthFinger', 'y_norm_cm')]
        ax = df.loc[:, ('HindPawRightFifthFinger', 'x_norm_cm')]
        #area_secondfinger_hindpawright_Fifthfinger
        df_copy = df.copy()
        df_copy.loc[:, ('area_HindPawRight')] = self.triangle_area(ax, ay, bx, by, cx, cy)
        df.loc[:, ('area_HindPawRight')] = df_copy.loc[(df[('all', 'exclude')] == False) & (df[('HindPawRight', 'exclude')] == False) & (df[('HindPawRightFifthFinger', 'exclude')] == False) & (df[('HindPawRightSecondFinger', 'exclude')] == False), ('area_HindPawRight')]
        print(df_copy.loc[(df[('all', 'exclude')] == False) & (df[('HindPawRight', 'exclude')] == False) & (df[('HindPawRightFifthFinger', 'exclude')] == False) & (df[('HindPawRightSecondFinger', 'exclude')] == False), ('area_HindPawRight')])
        
        #HindPawLeftSecondFinger
        cy = df.loc[:, ('HindPawLeftSecondFinger', 'y_norm_cm')]
        cx = df.loc[:, ('HindPawLeftSecondFinger', 'x_norm_cm')]
        #HindPawLeft
        by = df.loc[:, ('HindPawLeft', 'y_norm_cm')]
        bx = df.loc[:, ('HindPawLeft', 'x_norm_cm')]
        #HindPawLeftFifthFinger
        ay = df.loc[:, ('HindPawLeftFifthFinger', 'y_norm_cm')]
        ax = df.loc[:, ('HindPawLeftFifthFinger', 'x_norm_cm')]
        #area_secondfinger_hindpawLeft_Fifthfinger
        df_copy = df.copy()
        df_copy.loc[:, ('area_HindPawLeft')] = self.triangle_area(ax, ay, bx, by, cx, cy)
        df.loc[:, ('area_HindPawLeft')] = df_copy.loc[(df[('all', 'exclude')] == False) & (df[('HindPawLeft', 'exclude')] == False) & (df[('HindPawLeftFifthFinger', 'exclude')] == False) & (df[('HindPawLeftSecondFinger', 'exclude')] == False), ('area_HindPawLeft')]
        print(df_copy.loc[(df[('all', 'exclude')] == False) & (df[('HindPawLeft', 'exclude')] == False) & (df[('HindPawLeftFifthFinger', 'exclude')] == False) & (df[('HindPawLeftSecondFinger', 'exclude')] == False), ('area_HindPawLeft')])
        
        return df 

    
    def triangle_area(self, x1, y1, x2, y2, x3, y3):
        area = abs(0.5 * (((x2-x1)*(y3-y1))-((x3-x1)*(y2-y1))))    
        return area
        

In [8]:
class maze_corners(bottom_cam):   
    """creates an object representing the maze_corners"""
    def __init__(self, filepath):
        self.filepath = filepath
        cap = cv2.VideoCapture(self.filepath)
        self.ret, self.frame = cap.read()
        #self.results = {} #testen
        self.results = {'offset_x': 1280, 'offset_y': 327, 'length': 1183, 'width': 64, 'theta': 1.5481070465189704}
        cap.release()
        
    def rotate(self, xy, theta):
        cos_theta, sin_theta = math.cos(theta), math.sin(theta)
        return (xy[0] * cos_theta - xy[1] * sin_theta, xy[0] * sin_theta + xy[1] * cos_theta)

    def translate(self, xy, offset):
        return xy[0] + offset[0], xy[1] + offset[1]

    def f(self, x, y, length, width, degrees):
        offset = (x, y)
        corners = [(0, 0), (width, 0), (width, length), (0, length)]
        rotated_and_shifted_corners = [self.translate(self.rotate(xy, math.radians(degrees)), offset) for xy in corners]

        end_right_corner = list(rotated_and_shifted_corners[0]) + ['red']
        end_left_corner = list(rotated_and_shifted_corners[1]) + ['orange']
        start_left_corner = list(rotated_and_shifted_corners[2]) + ['cyan']
        start_right_corner = list(rotated_and_shifted_corners[3]) + ['green']

        fig = plt.figure(figsize=(18, 10))
        gs = fig.add_gridspec(2, 4)

        fig.add_subplot(gs[0:2, 0:2])
        plt.imshow(self.frame)
        plt.ylim(0,self.frame.shape[0])
        plt.xlim(0,self.frame.shape[1])

        if len(self.results.keys()) > 0:
            saved_current = 'saved'
        else:
            saved_current = 'missing'

        plt.title('current file: {} (analysis {})'.format(self.filepath, saved_current))

        l_corners = [start_right_corner, start_left_corner, end_right_corner, end_left_corner]

        for corner in l_corners:
            plt.scatter(corner[0], corner[1], c=corner[2], s=100)

        fig.add_subplot(gs[0, 2])
        plt.imshow(self.frame)
        plt.scatter(l_corners[0][0], l_corners[0][1], c=l_corners[0][2], s=100)
        plt.xlim(l_corners[0][0]-25, l_corners[0][0]+25)
        plt.ylim(l_corners[0][1]-25, l_corners[0][1]+25)
        plt.title('start right corner')

        fig.add_subplot(gs[0, 3])
        plt.imshow(self.frame)
        plt.scatter(l_corners[1][0], l_corners[1][1], c=l_corners[1][2], s=100)
        plt.xlim(l_corners[1][0]-25, l_corners[1][0]+25)
        plt.ylim(l_corners[1][1]-25, l_corners[1][1]+25)
        plt.title('start left corner')

        fig.add_subplot(gs[1, 2])
        plt.imshow(self.frame)
        plt.scatter(l_corners[2][0], l_corners[2][1], c=l_corners[2][2], s=100)
        plt.xlim(l_corners[2][0]-25, l_corners[2][0]+25)
        plt.ylim(l_corners[2][1]-25, l_corners[2][1]+25)
        plt.title('end right corner')

        fig.add_subplot(gs[1, 3])
        plt.imshow(self.frame)
        plt.scatter(l_corners[3][0], l_corners[3][1], c=l_corners[3][2], s=100)
        plt.xlim(l_corners[3][0]-25, l_corners[3][0]+25)
        plt.ylim(l_corners[3][1]-25, l_corners[3][1]+25)
        plt.title('end left corner')

        plt.show()

In [9]:
class gui():
    """represents the GUI"""
    main_tab = widgets.Tab()   
    tab_index = -1
    displayed = False
    
    class main_gui(main):
        """creates a main object as specified by the path input and recording modalities"""
        def __init__(self):
            self.path = ""
            folder_select = widgets.Button(description="Select folder", layout=widgets.Layout(width="auto"))
            folder_select.on_click(self.select_folder)
            
            select_recording_modalities = widgets.Label(value="Select recording modalities", layout=widgets.Layout(width="auto"))
            self.bottom_cam_check = widgets.Checkbox(value=False, description='Bottom Cam', layout=widgets.Layout(width="auto"))
            self.top_cam_check = widgets.Checkbox(value=False, description='Top Cam', disabled = True, layout=widgets.Layout(width="auto"))
            self.side_cam_check = widgets.Checkbox(value=False, description='Side Cam', disabled = True, layout=widgets.Layout(width="auto"))
            
            confirm_button = widgets.Button(description = "Confirm Settings", layout=widgets.Layout(width="auto"))
            confirm_button.on_click(self.confirm_settings)
            
            col0 = VBox([folder_select])
            col1 = VBox([select_recording_modalities, self.bottom_cam_check, self.top_cam_check, self.side_cam_check])
            col2 = VBox([confirm_button])
            box = HBox([col0, col1, col2])
            gui.main_tab.children = [box]
            gui.tab_index += 1
            gui.main_tab.set_title(gui.tab_index, "General Settings")
            display(gui.main_tab)
            gui.displayed = True
            gui.main_tab.selected_index = gui.tab_index

        def select_folder(self, b):
            root = Tk()
            root.withdraw()
            root.call('wm', 'attributes', '.', '-topmost', True)
            self.path = filedialog.askdirectory() + "/"
            display(self.path)        
            
        def confirm_settings(self, b):
            if self.path == "":
                display("Set the path before continuing!")
            else:
                gui.a = main(path = self.path, dict_cams_used= {"bottom_cam": self.bottom_cam_check.value, "top_cam": self.top_cam_check.value, "side_cam": self.side_cam_check.value}, gui=True)
                gui.a.all_information_given()
                if gui.a.all_files_there == True:
                    gui.subject_gui()
        
    class subject_gui(main):
        """allows to allocate subjects to groups"""
        def __init__(self):
            self.num_of_groups_dropdown = widgets.Dropdown(options=[1, 2, 3, 4, 5, 6], value=2, description='Choose, how many groups you have in your dataset', layout=widgets.Layout(width="auto"), style={'description_width': 'auto'})
            confirm_groups_button = widgets.Button(description='Confirm number of groups', layout=widgets.Layout(width="auto"))
            confirm_groups_button.on_click(self.name_groups)
            row0 = HBox([self.num_of_groups_dropdown, confirm_groups_button])
            box = VBox([row0])
            gui.main_tab.children += (box, ) 
            gui.tab_index += 1
            gui.main_tab.set_title(gui.tab_index, "Subjects to groups")
            gui.main_tab.selected_index = gui.tab_index
            
        def name_groups(self, b):
            self.num_of_groups = self.num_of_groups_dropdown.value
            self.l_group_texts = [widgets.Text(value = 'group {}'.format(n), description='Name of group {}'.format(n), layout=widgets.Layout(width="auto"), style = {'description_width': 'auto'}) for n in range (self.num_of_groups)]
            name_subjects_button = widgets.Button(description = "Confirm name of the groups", layout=widgets.Layout(width="auto"))
            name_subjects_button.on_click(self.subjects_to_groups)
            box = HBox([VBox(self.l_group_texts), name_subjects_button])
            gui.main_tab.children[gui.tab_index].children += (box,)

            
        def subjects_to_groups(self, b):
            l_subject_label = [widgets.Label(value=subject.subject_ID, layout=widgets.Layout(width="auto")) for subject in gui.a.l_subjects]
            self.l_group_toggle_buttons = [widgets.ToggleButtons(options = [text.value for text in self.l_group_texts], layout=widgets.Layout(width="auto")) for subject in range(len(gui.a.l_subjects))]
            continue_button = widgets.Button(description = "All subjects in the right group", layout=widgets.Layout(width="auto"))
            continue_button.on_click(self.next_step)
            col0 = VBox(l_subject_label)
            col1 = VBox(self.l_group_toggle_buttons)
            box = HBox([col0, col1, continue_button])
            gui.main_tab.children[gui.tab_index].children += (box,)
            
        def next_step(self, b):
            l_groups = [self.l_group_texts[n].value for n in range(len(self.l_group_texts))]
            gui.a.subjects_to_groups({subject.subject_ID:self.l_group_toggle_buttons[n].value for subject, n in zip (gui.a.l_subjects, range(len(gui.a.l_subjects)))}, l_groups)
            gui.a.get_maze_corners()
      
    class select_functions(main):
        """allows to select the target variables, that will be analysed"""
        def __init__(self):
            select_functions = widgets.Label(value="Select functions in which you're interested in", layout=widgets.Layout(width="auto"), style = {'description_width': 'auto'})
            self.anxiety_check = widgets.Checkbox(value=False, description='Anxiety', disabled = True, layout=widgets.Layout(width="auto"))
            self.parkinson_check = widgets.Checkbox(value=False, description='Parkinson', disabled = True, layout=widgets.Layout(width="auto"))
            self.catwalk_check = widgets.Checkbox(value=False, description='Catwalk', layout=widgets.Layout(width="auto"))
            
            self.l_checkboxes = [self.anxiety_check, self.parkinson_check, self.catwalk_check]

            confirm_selection_button = widgets.Button(description = "Confirm Selection", layout=widgets.Layout(width="auto"))
            confirm_selection_button.on_click(self.confirm_selection)

            col0 = VBox([select_functions, self.anxiety_check, self.parkinson_check, self.catwalk_check])
            col1 = VBox([confirm_selection_button])
            box = HBox([col0, col1])
            gui.main_tab.children += (box, ) 
            gui.tab_index += 1
            gui.main_tab.set_title(gui.tab_index, "Select Functions")
            gui.main_tab.selected_index = gui.tab_index

        def confirm_selection(self, b):
            l_selected_functions_values = [checkbox.value for checkbox in self.l_checkboxes]
            l_selected_functions_keys = [checkbox.description for checkbox in self.l_checkboxes]
            dict_selected_functions = {key:value for key,value in zip(l_selected_functions_keys,l_selected_functions_values)}
            gui.a.execute_functions(dict_selected_functions)

In [10]:
class maze_corner_gui(main):
    """creates the GUI for annotation maze corners, used in GUI as well as GUI-less usage"""
    def __init__(self):
        self.maze_corner_idx = 0
        self.actualize()
        self.clean_df = False
        self.create_gui()

    def on_load_next_button_click(self, b):
        if self.results_saved:
            if self.maze_corner_idx >= (len(main.l_maze_corners)-1):
                #check, whether all Maze Corners are saved to bc.mc.results
                if self.clean_df == False:
                    print("Maze Corners for all videos set.")
                    main.get_processed_dfs(main)
                    self.clean_df = True
                    print("Dataframes cleaned.")
                    if main.gui == True:
                        gui.select_functions()
                else: 
                    self.maze_corner_idx += 1
                    self.actualize()
        else:
            display("Please save the settings before continuing!")

    def on_load_previous_button_click(self, b):
        if self.maze_corner_idx <= 0:
            display("Index out of range! Index has been set to 0.")
            self.maze_corner_idx = 0
        else:
            if self.results_saved:
                self.maze_corner_idx -= 1
                self.actualize()
            else:
                display("Please save the settings before continuing!")

    def actualize(self):
        self.results_saved = False

    def on_save_button_click(self, b):
        main.l_maze_corners[self.maze_corner_idx].results["offset_x"] = self.interactive_plot.children[0].value
        main.l_maze_corners[self.maze_corner_idx].results["offset_y"] = self.interactive_plot.children[1].value
        main.l_maze_corners[self.maze_corner_idx].results["length"] = self.interactive_plot.children[2].value
        main.l_maze_corners[self.maze_corner_idx].results["width"] = self.interactive_plot.children[3].value
        main.l_maze_corners[self.maze_corner_idx].results["theta"] = math.radians(self.interactive_plot.children[4].value)
        self.results_saved = True
        print(main.l_maze_corners[self.maze_corner_idx].results)

    def create_gui(self):
        height, width = main.l_maze_corners[self.maze_corner_idx].frame.shape[0], main.l_maze_corners[self.maze_corner_idx].frame.shape[1]#replace slider with int, since slider are very slow
        slider_x = widgets.IntSlider(value=300, min=0, max=width, step=1, description='x offset', continuous_update=False)
        slider_y = widgets.IntSlider(value=5, min=0, max=height, step=1, description='y offset', continuous_update=False)
        slider_length = widgets.IntSlider(value=height/2, min=0, max=width, step=1, continuous_update=False)
        slider_width = widgets.IntSlider(value=width/20, min=0, max=height/7, step=1, continuous_update=False)
        slider_degrees = widgets.FloatSlider(value=0, min=0, max=180, step=0.1, continuous_update=False)

        self.interactive_plot = interactive(main.l_maze_corners[self.maze_corner_idx].f, x=slider_x, y=slider_y, length=slider_length, width=slider_width, degrees=slider_degrees)

        self.interactive_plot.children[-1].layout.height = '600px'

        load_next_button = widgets.Button(description="Load next file", style = {'description_width': 'auto'})
        save_button = widgets.Button(description="Save settings", style = {'description_width': 'auto'})
        load_previous_button = widgets.Button(description="Load previous file", style = {'description_width': 'auto'})

        load_next_button.on_click(self.on_load_next_button_click)
        load_previous_button.on_click(self.on_load_previous_button_click)
        save_button.on_click(self.on_save_button_click)

        col0 = VBox([load_next_button, save_button])
        col1 = VBox([self.interactive_plot.children[0], self.interactive_plot.children[1]])
        col2 = VBox([self.interactive_plot.children[2], self.interactive_plot.children[3]])
        col3 = VBox([self.interactive_plot.children[4], load_previous_button])
        row0 = HBox([col0, col1, col2, col3])
        box = VBox([row0, self.interactive_plot.children[-1]])

        gui.main_tab.children += (box, ) 
        gui.tab_index += 1
        gui.main_tab.set_title(gui.tab_index, "Set Maze Corners")
        gui.main_tab.selected_index = gui.tab_index
    
class stats_gui(main):
    """allows to choose and individualize the stats"""
    output_path = ""

    def __init__(self, dict_selected_functions):
        self.dict_selected_functions = dict_selected_functions
        self.select_stats_dropdown = widgets.RadioButtons(options=["basic", "all", "select"], value="basic", description = "Choose, which statistics you want to plot", layout=widgets.Layout(width="auto"), style = {'description_width': 'auto'})

        confirm_button = widgets.Button(description = "Confirm", layout=widgets.Layout(width="auto"))
        confirm_button.on_click(self.confirm)

        enter_path = widgets.Button(description="Select output folder", layout=widgets.Layout(width="auto"))
        enter_path.on_click(self.select_output_path)

        select_ind_var_label = widgets.Label(value="Select independent variable", layout=widgets.Layout(width="auto"))
        self.select_ind_variable = widgets.Dropdown(options = ["group_ID", "paradigm"], layout=widgets.Layout(width="auto"))
        select_hue_label = widgets.Label(value="Select hue", layout=widgets.Layout(width="auto"))
        self.select_hue = widgets.Dropdown(options = ["subject_ID", "paradigm", "group_ID"], layout=widgets.Layout(width="auto"))

        col0 = VBox([self.select_stats_dropdown, confirm_button])
        col1 = VBox([select_ind_var_label, self.select_ind_variable])
        col2 = VBox([select_hue_label, self.select_hue])
        col3 = VBox ([enter_path])
        row0 = HBox([col0, col1, col2, col3])
        box = VBox([row0])
        gui.main_tab.children += (box, ) 
        gui.tab_index += 1
        gui.main_tab.set_title(gui.tab_index, "Select Statistics")
        gui.main_tab.selected_index = gui.tab_index

    def select_output_path(self, b):
        root = Tk()
        root.withdraw()
        root.call('wm', 'attributes', '.', '-topmost', True)
        self.output_path = filedialog.askdirectory() + "/"
        display(self.output_path)        

    def confirm(self, b):
        if self.select_stats_dropdown.value == "select":
            self.select_data_col()
        else:
            if self.select_stats_dropdown.value == "basic":
                self.l_selected_data_col = []
                if self.dict_selected_functions["Anxiety"]:
                    self.l_selected_data_col.extend(['count_freezing_bouts', 'percentage_of_time_spent_freezing', 'mean_freezing_bouts_y_position'])
                if self.dict_selected_functions["Parkinson"]:
                    self.l_selected_data_col.extend(['mean_gait_disruption_bouts_y_position_all'])
                if self.dict_selected_functions["Catwalk"]:
                    self.l_selected_data_col.extend(['mean_distance_secondtoe_hindpawright', 'mean_distance_secondtoe_hindpawleft', 'mean_area_hindpawright', 'mean_area_hindpawleft'])
            elif self.select_stats_dropdown.value == "all":
                self.l_selected_data_col = [key for key in main.d_data.keys() if key not in set(['subject_ID', 'group_ID', 'paradigm', 'trialnumber'])]
            l_selected_data_col_dict = []
            for data_col in self.l_selected_data_col:
                dict_stats = {}
                dict_stats["data_col"] = data_col
                dict_stats["independent_variable"] =  self.select_ind_variable.value
                dict_stats["hue"] =  self.select_hue.value
                l_selected_data_col_dict.append(dict_stats)
            main.calculate_stats(main, self.output_path, l_selected_data_col_dict)

    def select_data_col(self):
        confirm_button = widgets.Button(description = "Confirm", layout=widgets.Layout(width="auto"))
        confirm_button.on_click(self.confirm_selected_data_col)
        select_data_key_label = widgets.Label(value="Select Data Column", layout=widgets.Layout(width="auto"))
        select_ind_var_label = widgets.Label(value="Select independent variable", layout=widgets.Layout(width="auto"))
        select_hue_label = widgets.Label(value="Select hue", layout=widgets.Layout(width="auto"))

        l_grid_children = [select_data_key_label, select_ind_var_label, select_hue_label]
        for key in main.d_data.keys(): 
            if key not in set(['subject_ID', 'group_ID', 'paradigm', 'trialnumber']):
                l_grid_children.append(widgets.Checkbox(value=False, description=key, layout=widgets.Layout(width="auto")))
                l_grid_children.append(widgets.Dropdown(options = ["group_ID", "paradigm"], value='group_ID', layout=widgets.Layout(width="auto")))
                l_grid_children.append(widgets.Dropdown(options = ["subject_ID", "paradigm", "group_ID"], value='subject_ID', layout=widgets.Layout(width="auto")))

        enter_path = widgets.Button(description="Select output folder", layout=widgets.Layout(width="auto"))
        enter_path.on_click(self.select_output_path)

        col3 = VBox([enter_path, confirm_button]) 

        self.grid = widgets.GridBox(l_grid_children, layout=widgets.Layout(grid_template_columns="repeat(3, auto)"))

        box = HBox([self.grid, col3])
        gui.main_tab.children[gui.tab_index].children += (box,)
        gui.main_tab.children[gui.tab_index].children[0].children = (gui.main_tab.children[gui.tab_index].children[0].children[0], )

    def confirm_selected_data_col(self, b):
        l_selected_data_col_dict = []
        for n in range(len(self.grid.children)):
            if n>2 & n%3 == 0:
                if self.grid.children[n].value == True:
                    dict_stats = {}
                    dict_stats["data_col"] = self.grid.children[n].description
                    dict_stats["independent_variable"] =  self.grid.children[n+1].value
                    dict_stats["hue"] =  self.grid.children[n+2].value
                    l_selected_data_col_dict.append(dict_stats)
        main.calculate_stats(main, self.output_path, l_selected_data_col_dict)

In [11]:
class stats(main):
    """creates stats and output plots/.csv files as specified in the stats gui"""
    def __init__(self):
        pass
    
    def position_stats(self, data_col, independent_variable, hue, output_path):
        l_columns = ['group_ID', 'subject_ID', 'paradigm']

        dataframe = main.dataframe.loc[:, [data_col] + l_columns]
        dataframe.reset_index(inplace=True, drop=True)
        
        plt.figure(figsize=(7,9), facecolor='white')
        
        sns.violinplot(data=dataframe, y="paradigm", x=data_col, fliersize=0, orient='h', hue=independent_variable)
        sns.stripplot(data=dataframe, y="paradigm", x=data_col, orient='h', color='k', hue=independent_variable, dodge=True, alpha=0.3)
        plt.vlines(x=35, ymin=0.5, ymax=3.5, color='magenta', linestyle='dashed')

        if output_path != "":
            figname = 'differences between {} for {}.png'.format(data_col, independent_variable)
            plt.savefig(output_path + figname, dpi=300)
            csv_name = figname.replace('png', 'csv')
            dataframe.to_csv(output_path + csv_name)
    
        plt.xlim(0, 75)
        plt.legend(loc='center right')
        plt.title(data_col + " per " + independent_variable)
        plt.show()
        #plt.close()
        
    def total_count_stats(self, data_col, independent_variable, hue, output_path):
        l_columns = ['group_ID', 'subject_ID', 'paradigm']
        
        dataframe = main.dataframe.loc[:, [data_col] + l_columns].copy()
        dataframe.reset_index(inplace=True, drop=True)

        plt.figure(figsize=(7,4), facecolor='white')
        
        sns.boxplot(data=dataframe, y="paradigm", x=data_col, hue=independent_variable, fliersize=0)
        sns.stripplot(data=dataframe, y="paradigm", x=data_col, hue=independent_variable, dodge=True, color='k')
        
        if output_path != "":
            figname = 'differences between {} for {}.png'.format(data_col, independent_variable)
            plt.savefig(output_path + figname, dpi=300)
            csv_name = figname.replace('png', 'csv')
            dataframe.to_csv(output_path + csv_name)

        plt.ylim(0)
        plt.xlim(-0.5,5.5)
        #plt.legend('')
        plt.title(data_col + " per " + independent_variable)
        plt.show()
        #plt.close()

Use GUI for the Code:

In [ ]:
gui.main_gui()

GUI-less usage (as much as possible) of the Code:

In [12]:
project = main(path = "F:/Konstantin/data/Inken/Dummy/", gui=False)

Maus16_post-1xIgG_20220415
Maus16_post-OP3_20220407
Maus16_post-OP_20220414
Maus16_post3xIgG_20220417
Maus16_post6xIgG_20220420
Maus16_prä-OP1_20220405
Maus16_prä-OP2_20220306


In [14]:
project.subjects_to_groups(["control", "experimental"])

In [15]:
project.execute_functions()

Series([], Name: angle_secondfinger_hindpawright_hindkneeright, dtype: float64)
Series([], Name: angle_hindpawright_hindkneeright_CoG, dtype: float64)
Series([], Name: angle_secondfinger_hindpawleft_hindkneeleft, dtype: float64)
Series([], Name: angle_hindpawleft_hindkneeleft_CoG, dtype: float64)
0      4.996181
21     3.180556
35     1.285494
41     1.368525
47     1.323118
         ...   
793    1.630484
795    1.577875
796    1.437266
818    2.487539
824    1.284197
Name: distance_secondfinger_hindpawright, Length: 210, dtype: float64
0      1.323594
18     1.817836
21     2.014272
24     1.917999
25     1.917999
         ...   
818    1.659907
819    1.654223
820    1.803669
822    1.828063
823    1.848839
Name: distance_secondfinger_hindpawleft, Length: 495, dtype: float64
0      2.599688
137    1.774528
138    0.185703
359    0.229732
385    0.271538
388    0.251536
414    0.239209
551    0.297739
567    0.355159
592    0.309845
600    0.304675
729    0.310159
760    0.310113
818

220      78.533273
231      23.095306
232      26.868808
250      92.099576
251      92.099576
255      92.156770
256      92.156770
271      27.180780
272      91.684723
276      86.000460
277      86.000460
280     161.444571
281      75.392226
287      74.863991
288     154.942162
300      91.329847
301      91.325209
313      95.495022
331     119.390614
332      94.835596
382      95.697152
383     198.423245
386     200.207697
387     196.437781
396      70.730668
469     113.819721
536      74.595228
545      92.032789
546      94.425288
557     220.819323
558     197.998350
579     131.044571
588     105.278668
595      95.917732
596     115.586801
622      91.051487
623     150.021458
660      76.526095
675      75.659976
677      80.566076
678      84.993325
680      87.029911
685      78.055144
1021    302.517491
1022    303.520598
1039    259.649362
1040    258.793223
1042    261.836391
1043    263.781610
1045    263.936847
1046    263.299497
Name: angle_secondfinger_hindpa

Tab()